In [76]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt

import logging
from time import sleep
import stomp

In [81]:
def listen(secs):
    NETWORK_RAIL_AUTH = ('username', 'password')

    class Listener(object):

        def __init__(self, mq):
            self._mq = mq
            self.msg_list = []

        def on_message(self, headers, message):
            self.msg_list.append(message)
            self._mq.ack(id=headers['message-id'], subscription=headers['subscription'])

    mq = stomp.Connection(host_and_ports=[('datafeeds.networkrail.co.uk', 61618)],
                          keepalive=True,
                          vhost='datafeeds.networkrail.co.uk',
                          heartbeats=(10000, 5000))

    Listener_obj = Listener(mq)

    mq.set_listener('', Listener_obj)

    mq.connect(username=NETWORK_RAIL_AUTH[0],
               passcode=NETWORK_RAIL_AUTH[1],
               wait=True)

    mq.subscribe('/topic/TD_WESS_SIG_AREA', 'test-td', ack='client-individual')

    sleep(secs) # secs

    mq.disconnect()

    output = Listener_obj.msg_list

    return output

In [70]:
output = listen(20)

keepalive: unable to detect any implementation, DISABLED!


[{"CT_MSG":{"time":"1589335868000","area_id":"FH","msg_type":"CT","report_time":"0254"}},{"CT_MSG":{"time":"1589335868000","area_id":"BP","msg_type":"CT","report_time":"0217"}},{"CT_MSG":{"time":"1589335869000","area_id":"BS","msg_type":"CT","report_time":"2157"}}]
[{"CA_MSG":{"to":"0510","time":"1589335872000","area_id":"BE","msg_type":"CA","from":"0514","descr":"428I"}},{"CT_MSG":{"time":"1589335874000","area_id":"HT","msg_type":"CT","report_time":"0306"}},{"CT_MSG":{"time":"1589335876000","area_id":"EH","msg_type":"CT","report_time":"0311"}},{"SF_MSG":{"time":"1589335875000","area_id":"EH","address":"3F","msg_type":"SF","data":"00"}}]
[{"CT_MSG":{"time":"1589335876000","area_id":"SU","msg_type":"CT","report_time":"0310"}},{"SF_MSG":{"time":"1589335878000","area_id":"BE","address":"35","msg_type":"SF","data":"AF"}}]
[{"CA_MSG":{"to":"0962","time":"1589335885000","area_id":"SU","msg_type":"CA","from":"0964","descr":"662I"}}]


In [116]:
pd.concat([pd.DataFrame(msg).T for o in output for msg in json.loads(o)])

,area_id,msg_type,report_time,time,descr,from,to,address,data
CT_MSG,FH,CT,0254,1589335868000,NaN,NaN,NaN,NaN,NaN
CT_MSG,BP,CT,0217,1589335868000,NaN,NaN,NaN,NaN,NaN
CT_MSG,BS,CT,2157,1589335869000,NaN,NaN,NaN,NaN,NaN
CA_MSG,BE,CA,NaN,1589335872000,428I,0514,0510,NaN,NaN
CT_MSG,HT,CT,0306,1589335874000,NaN,NaN,NaN,NaN,NaN
CT_MSG,EH,CT,0311,1589335876000,NaN,NaN,NaN,NaN,NaN
SF_MSG,EH,SF,NaN,1589335875000,NaN,NaN,NaN,3F,00
CT_MSG,SU,CT,0310,1589335876000,NaN,NaN,NaN,NaN,NaN
SF_MSG,BE,SF,NaN,1589335878000,NaN,NaN,NaN,35,AF
CA_MSG,SU,CA,NaN,1589335885000,662I,0964,0962,NaN,NaN


In [114]:
def parser(output):
    df_list = []
    for o in output:
        j = json.loads(o)
        for msg in j:
            df = pd.DataFrame(msg).T
            df_list.append(df)

    return pd.concat(df_list)

In [115]:
msgs_df = parser(output)
msgs_df

,area_id,msg_type,report_time,time,descr,from,to,address,data
CT_MSG,FH,CT,0254,1589335868000,NaN,NaN,NaN,NaN,NaN
CT_MSG,BP,CT,0217,1589335868000,NaN,NaN,NaN,NaN,NaN
CT_MSG,BS,CT,2157,1589335869000,NaN,NaN,NaN,NaN,NaN
CA_MSG,BE,CA,NaN,1589335872000,428I,0514,0510,NaN,NaN
CT_MSG,HT,CT,0306,1589335874000,NaN,NaN,NaN,NaN,NaN
CT_MSG,EH,CT,0311,1589335876000,NaN,NaN,NaN,NaN,NaN
SF_MSG,EH,SF,NaN,1589335875000,NaN,NaN,NaN,3F,00
CT_MSG,SU,CT,0310,1589335876000,NaN,NaN,NaN,NaN,NaN
SF_MSG,BE,SF,NaN,1589335878000,NaN,NaN,NaN,35,AF
CA_MSG,SU,CA,NaN,1589335885000,662I,0964,0962,NaN,NaN


In [79]:
ca_msgs = msgs_df[msgs_df.descr.notnull()]
ca_msgs

,area_id,msg_type,report_time,time,descr,from,to,address,data
CA_MSG,BE,CA,NaN,1589335872000,428I,0514,0510,NaN,NaN
CA_MSG,SU,CA,NaN,1589335885000,662I,0964,0962,NaN,NaN


In [230]:
def listen_and_save(secs):
    output = listen(secs)
    msgs_df = parser(output)
    
    dtype = ['category']*len(msgs_df.columns)
    dtype[3] = int
    
    columns = msgs_df.columns
    parse = {key: value for key, value in zip(columns, dtype)}
    msgs_df = msgs_df.astype(parse)
    msgs_df['time'] = pd.to_datetime(msgs_df['time'], unit='ns', utc = True)
    
    return msgs_df, msgs_df[msgs_df['descr'].notnull()]

In [231]:
from datetime import datetime

In [233]:
msgs_df, ca_msgs = listen_and_save(60)
display(msgs_df)
display(ca_msgs)

keepalive: unable to detect any implementation, DISABLED!


,area_id,msg_type,report_time,time,descr,from,to
CT_MSG,HT,CT,0354,1970-01-01 00:26:29.338787+00:00,NaN,NaN,NaN
CT_MSG,WI,CT,0401,1970-01-01 00:26:29.338796+00:00,NaN,NaN,NaN
CT_MSG,GD,CT,0355,1970-01-01 00:26:29.338798+00:00,NaN,NaN,NaN
CT_MSG,SU,CT,0359,1970-01-01 00:26:29.338799+00:00,NaN,NaN,NaN
CT_MSG,ZG,CT,0400,1970-01-01 00:26:29.338799+00:00,NaN,NaN,NaN
CT_MSG,ZY,CT,0400,1970-01-01 00:26:29.338800+00:00,NaN,NaN,NaN
CT_MSG,ZB,CT,1451,1970-01-01 00:26:29.338800+00:00,NaN,NaN,NaN
CT_MSG,BE,CT,0400,1970-01-01 00:26:29.338808+00:00,NaN,NaN,NaN
CT_MSG,BP,CT,0217,1970-01-01 00:26:29.338808+00:00,NaN,NaN,NaN
CT_MSG,FH,CT,0343,1970-01-01 00:26:29.338808+00:00,NaN,NaN,NaN


,area_id,msg_type,report_time,time,descr,from,to
CA_MSG,SU,CA,NaN,1970-01-01 00:26:29.338815+00:00,479I,0177,0181
CA_MSG,BE,CA,NaN,1970-01-01 00:26:29.338815+00:00,479I,W177,W181


In [234]:
msgs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, CT_MSG to CT_MSG
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   area_id      27 non-null     category           
 1   msg_type     27 non-null     category           
 2   report_time  25 non-null     category           
 3   time         27 non-null     datetime64[ns, UTC]
 4   descr        2 non-null      category           
 5   from         2 non-null      category           
 6   to           2 non-null      category           
dtypes: category(6), datetime64[ns, UTC](1)
memory usage: 2.1+ KB
